In [ ]:
import os, logging
import requests, json
import re, sys, subprocess, time

LOG = logging.getLogger()

EXCEPTION_COMMAND = """osascript -e 'tell app "Finder" to display dialog "{}"'"""
BATT_INFO_COMMAND = "pmset -g batt"
LED_JSON_URL = "http://192.168.0.140/json/state"
LED_PRESET_URL = "http://192.168.0.140/win&PL={}"
TEMP_STORAGE_DIR = "/private/tmp/wled_subprocesses/{}"

LED_PRESET_MAP = {
    "charging" : 6,
    "charged"  : 2,
    "low battery" : 5}


def call_err_and_quit(err):
    command = EXCEPTION_COMMAND.format(err)
    subprocess.run([command], shell=True)
    sys.exit()


def get_battery_data():
    LOG.info("Getting battery information")
    batt_info = re.findall("\\\t(.*)", subprocess.run([BATT_INFO_COMMAND], shell=True, capture_output=True, text=True).stdout)
    if (len(batt_info) != 1
        or len(batt_info[0].split(";")) != 3
        or "%" not in batt_info[0].split(";")[0]):

        err = f"""Invalid results for `pmset -g batt`: \n\n{out.replace('"', "").replace("'", "")}"""
        call_err_and_quit(err)

    batt_info  = batt_info[0].split(";")
    batt_data = {
        "level"  : int(batt_info[0].replace("%", "")),
        "status" : batt_info[1].strip()}
    LOG.info(f"Battery info : {json.dumps(batt_data, indent=2)}")
    return batt_data


def get_curr_state():
    header = {
        "content-type" : "application/json"}
    return requests.get(LED_JSON_URL, headers=header).json()


def save_curr_state(info):
    if not os.path.isdir(TEMP_STORAGE_DIR.format("")):
        os.mkdir(TEMP_STORAGE_DIR.format(""))

    _path = TEMP_STORAGE_DIR.format("curr_state.json")
    with open(_path, "w") as f:
        f.write(json.dumps(info, indent=2))
    return _path


def set_led_preset(preset_num):
    assert isinstance(preset_num, int)
    requests.post(LED_PRESET_URL.format(preset_num))


def revert_to_saved_state(prest_path):
    with open(prest_path, "r") as f:
        preset = f.read()
    if preset:
        header = {
            "content-type" : "application/json"}
        requests.post(LED_JSON_URL, headers=header, json=json.loads(preset)).json()
    else:
        raise ValueError("Preset file empty!")


def charged_routine():
    _changed_state = False
    battery_info = get_battery_data()
    while battery_info["level"] > 90 and battery_info["status"] in ["charging", "charged"]:
        # Change state only if it is different. Ignore if same
        if not _changed_state:
            set_led_preset(2)
            _changed_state = True
        time.sleep(600)
        battery_info = get_battery_data()
        # denotes that the led behavior was `changed`
        return  True
    # denotes that the led behavior was `not changed`
    return False


def low_battery_routine():
    _changed_state = False
    battery_info = get_battery_data()
    while battery_info["level"] < 30 and battery_info["status"] in ["discharging"]:
        # Change state only if it is different. Ignore if same
        if not _changed_state:
            set_led_preset(2)
            _changed_state = True
        time.sleep(600)
        battery_info = get_battery_data()
        # denotes that the led behavior was `changed`
        return  True
    # denotes that the led behavior was `not changed`
    return False


def main():
    LOG.info("Starting script execution...")

    LOG.info("Getting LED current state")
    curr_led_state = get_curr_state()
    LOG.info(f"LED Current state {json.dumps(curr_led_state, indent=2)}")
    saved_state_path = save_curr_state(curr_led_state)

    LOG.info("Running CHARGED routine")
    if charged_routine():
        revert_to_saved_state(saved_state_path)

    LOG.info("Running LOW BATTERY routine")
    if low_battery_routine():
        revert_to_saved_state(saved_state_path)


main()